# TLDR
Goal of this notebook is to understand why LMC model fitting is so slow

In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
# file_dir = os.path.dirname(__file__)
# sys.path.append(file_dir)
sys.path.append('/home/yz685/low_rank_BOPE')
sys.path.append('/home/yz685/low_rank_BOPE/low_rank_BOPE')
import warnings
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import torch
import time
from collections import defaultdict
from low_rank_BOPE.bope_class import BopeExperiment
from low_rank_BOPE.test_problems.synthetic_problem import make_controlled_coeffs, make_problem, generate_principal_axes, LinearUtil
from low_rank_BOPE.src.diagnostics import check_outcome_model_fit, check_util_model_fit

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Experiment 1; Outcome model quality vs data size, for fixed outcome dimensionality

In [2]:
outcome_dim = 20
rank = 1
input_dim = 1
noise_std = 0.05

full_axes = generate_principal_axes(
    output_dim=outcome_dim,
    num_axes=outcome_dim,
    seed = 0,
    dtype=torch.double
)

beta = make_controlled_coeffs(
    full_axes=full_axes,
    latent_dim=rank,
    alpha=1,
    n_reps = 1,
    dtype=torch.double
).transpose(-2, -1)

util_func = LinearUtil(beta=beta)

true_axes = full_axes[: rank]

problem = make_problem(
    input_dim = input_dim, 
    outcome_dim = outcome_dim,
    noise_std = noise_std,
    num_initial_samples = input_dim*outcome_dim,
    true_axes = true_axes,
    PC_lengthscales = [0.5]*rank,
    PC_scaling_factors = [2]
)


/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/botorch/test_functions/base.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "bounds", torch.tensor(self._bounds, dtype=torch.float).transpose(-1, -2)


In [10]:
def run_fitting_helper(
    methods=["st", "pca", "lmc"], 
    datasizes=[20,50,100], 
    output_path="/home/yz685/low_rank_BOPE/experiments/synthetic/test_model_fit_",
    outcome_dim=20,
    trial_idx = 101
):

    fitting_time_dict = defaultdict(dict)
    mse_dict = defaultdict(dict)

    exp = BopeExperiment(
        problem, 
        util_func, 
        methods = methods,
        pe_strategies = ["EUBO-zeta"],
        trial_idx = trial_idx,
        output_path = output_path,
    )

    for datasize in datasizes:
        print(f"========Running datasize = {datasize}========")
        exp.generate_random_experiment_data(n=datasize, compute_util = False)
        for method in methods:
            print(f"======Running method {method}=======")
            start_time = time.time()
            exp.fit_outcome_model(method)
            model_fitting_time = time.time() - start_time
            mse = check_outcome_model_fit(exp.outcome_models_dict[method], exp.problem, n_test=1000)

            fitting_time_dict[datasize][method] = model_fitting_time
            mse_dict[datasize][method] = mse

            print(f"Fitting time {model_fitting_time} sec; mse {mse}")

            save_path = output_path + f'outcome_dim={outcome_dim}/'
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            torch.save(fitting_time_dict, save_path + f'fitting_time_dict_trial={trial_idx}.th')
            torch.save(mse_dict, save_path + f'mse_dict_trial={trial_idx}.th')
    
    return fitting_time_dict, mse_dict



In [11]:
run_fitting_helper()

self.methods,  ['pca', 'st', 'lmc']
========Running datasize = 20========
======Running method st=======
Fitting outcome model using st
torch.Size([1000])
torch.Size([1000])
Fitting time 27.020501136779785 sec; mse 12.750066430199984
======Running method pca=======
Fitting outcome model using pca


/home/yz685/low_rank_BOPE/low_rank_BOPE/src/transforms.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.axes_learned = torch.tensor(axes_learned, **tkwargs)
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


torch.Size([1000])
amount of variance explained by 4 axes: 0.9574552041365618
torch.Size([1000])
Fitting time 270.1690630912781 sec; mse 6.510822195961352
======Running method lmc=======
Fitting outcome model using lmc
torch.Size([1000])
torch.Size([1000])
Fitting time 823.4178478717804 sec; mse 15191.1721277153
========Running datasize = 50========
======Running method st=======
Fitting outcome model using st
torch.Size([1000])
torch.Size([1000])
Fitting time 32.55560636520386 sec; mse 5.228767795338513
======Running method pca=======
Fitting outcome model using pca


/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


torch.Size([1000])
amount of variance explained by 4 axes: 0.9574552041365618
torch.Size([1000])
Fitting time 78.75699949264526 sec; mse 2.8767013755194073
======Running method lmc=======
Fitting outcome model using lmc
torch.Size([1000])
torch.Size([1000])
Fitting time 1266.670479297638 sec; mse 14765.268391159507
========Running datasize = 100========
======Running method st=======
Fitting outcome model using st
torch.Size([1000])
torch.Size([1000])
Fitting time 45.45489501953125 sec; mse 5.167503228065888
======Running method pca=======
Fitting outcome model using pca


/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


torch.Size([1000])
amount of variance explained by 4 axes: 0.9574552041365618
torch.Size([1000])
Fitting time 29.37682294845581 sec; mse 3.0122810174294283
======Running method lmc=======
Fitting outcome model using lmc
torch.Size([1000])
torch.Size([1000])
Fitting time 4081.128522634506 sec; mse 18261.422503947782


(defaultdict(dict,
             {20: {'st': 27.020501136779785,
               'pca': 270.1690630912781,
               'lmc': 823.4178478717804},
              50: {'st': 32.55560636520386,
               'pca': 78.75699949264526,
               'lmc': 1266.670479297638},
              100: {'st': 45.45489501953125,
               'pca': 29.37682294845581,
               'lmc': 4081.128522634506}}),
 defaultdict(dict,
             {20: {'st': 12.750066430199984,
               'pca': 6.510822195961352,
               'lmc': 15191.1721277153},
              50: {'st': 5.228767795338513,
               'pca': 2.8767013755194073,
               'lmc': 14765.268391159507},
              100: {'st': 5.167503228065888,
               'pca': 3.0122810174294283,
               'lmc': 18261.422503947782}}))

In [14]:
test_posterior_mean = torch.randn((4,3))
test_Y = test_posterior_mean.detach().clone()

print(test_Y)
print(torch.sum((test_posterior_mean - test_Y) ** 2, dim=1))

print(torch.sum(test_Y**2, dim=1))

mse_rel = torch.sum((test_posterior_mean - test_Y) ** 2, dim=1) / torch.sum(test_Y**2, dim=1)

print(mse_rel.shape)
print(torch.sqrt(mse_rel))
print(torch.sqrt(mse_rel).mean(axis=0).item())


tensor([[ 0.9567, -0.2218, -0.6547],
        [ 0.8627, -0.9438, -2.0737],
        [-0.3257,  0.4709,  1.3286],
        [-0.9000,  2.2574, -1.4764]])
tensor([0., 0., 0., 0.])
tensor([1.3931, 5.9353, 2.0931, 8.0858])
torch.Size([4])
tensor([0., 0., 0., 0.])
0.0


In [35]:
exp.fit_outcome_model("pca")

Fitting outcome model using pca


/home/yz685/low_rank_BOPE/low_rank_BOPE/src/transforms.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.axes_learned = torch.tensor(axes_learned, **tkwargs)


torch.Size([1000])
amount of variance explained by 3 axes: 0.9541438122197349


/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


In [36]:
check_outcome_model_fit(exp.outcome_models_dict["pca"], exp.problem, n_test=1000)

torch.Size([1000])


15.120272225107907

In [9]:
start_time = time.time()
exp.fit_outcome_model("lmc")
print(f'LMC model fitting used {time.time() - start_time} seconds')

Fitting outcome model using lmc
LMC model fitting used 235.67349195480347 seconds


In [10]:
check_outcome_model_fit(exp.outcome_models_dict["lmc"], exp.problem, n_test=1000)

torch.Size([1000])


23516.61878909699

In [28]:
exp.outcome_models_dict["lmc"].__dict__

{'_num_outputs': 20,
 'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict([(52,
               <torch.nn.modules.module._WrappedHook at 0x7f2cf4675040>)]),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('likelihood',
               MultitaskGaussianLikelihood(
                 (raw_task_noises_constraint): GreaterThan(1.000E-04)
                 (raw_noise_constraint): GreaterThan(1.000E-04)
               )),
              ('mean_module',
               MultitaskMean(
                 (base_means): ModuleList(
                   (0): ConstantMean()
                   (1): ConstantMean()
                   (2): ConstantMean()
                   (3): ConstantMean()
               

In [37]:
start_time = time.time()
exp.fit_outcome_model("st")
print(f'ST model fitting used {time.time() - start_time} seconds')

Fitting outcome model using st
torch.Size([1000])
ST model fitting used 29.51462411880493 seconds


In [38]:
check_outcome_model_fit(exp.outcome_models_dict["st"], exp.problem, n_test=1000)

torch.Size([1000])


24.674626763608078